In [12]:
import numpy
# scipy.special for the sigmoid function expit()
import scipy.special
import matplotlib.pyplot as matplot
%matplotlib inline 

In [13]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [14]:
# neural network class definition
class neuralNetwork:
    
    
    # initialise the neural network
    def __init__(self, inputnodes, hiddennodes, outputnodes, learningrate):
        # set number of nodes in each input, hidden, output layer
        self.inodes = inputnodes
        self.hnodes = hiddennodes
        self.onodes = outputnodes
        
        # link weight matrices, wih and who
        # weights inside the arrays are w_i_j, where link is from node i to node j in the next layer
        # w11 w21
        # w12 w22 etc 
        self.wih = numpy.random.normal(0.0, pow(self.inodes, -0.5), (self.hnodes, self.inodes))
        self.who = numpy.random.normal(0.0, pow(self.hnodes, -0.5), (self.onodes, self.hnodes))

        # learning rate
        self.lr = learningrate
        
        # activation function is the sigmoid function
        self.activation_function = lambda x: scipy.special.expit(x)
        
        pass

    
    # train the neural network
    def train(self, inputs_list, targets_list):
        # convert inputs list to 2d array
        inputs = numpy.array(inputs_list, ndmin=2).T
        targets = numpy.array(targets_list, ndmin=2).T
        
        # calculate signals into hidden layer
        hidden_inputs = numpy.dot(self.wih, inputs)
        # calculate the signals emerging from hidden layer
        hidden_outputs = self.activation_function(hidden_inputs)
        
        # calculate signals into final output layer
        final_inputs = numpy.dot(self.who, hidden_outputs)
        # calculate the signals emerging from final output layer
        final_outputs = self.activation_function(final_inputs)
        
        # output layer error is the (target - actual)
        output_errors = targets - final_outputs
        # hidden layer error is the output_errors, split by weights, recombined at hidden nodes
        hidden_errors = numpy.dot(self.who.T, output_errors) 
        
        # update the weights for the links between the hidden and output layers
        self.who += self.lr * numpy.dot((output_errors * final_outputs * (1.0 - final_outputs)), numpy.transpose(hidden_outputs))
        
        # update the weights for the links between the input and hidden layers
        self.wih += self.lr * numpy.dot((hidden_errors * hidden_outputs * (1.0 - hidden_outputs)), numpy.transpose(inputs))
        
        pass

    
    # query the neural network
    def query(self, inputs_list):
        # convert inputs list to 2d array
        inputs = numpy.array(inputs_list, ndmin=2).T
        
        # calculate signals into hidden layer
        hidden_inputs = numpy.dot(self.wih, inputs)
        # calculate the signals emerging from hidden layer
        hidden_outputs = self.activation_function(hidden_inputs)
        
        # calculate signals into final output layer
        final_inputs = numpy.dot(self.who, hidden_outputs)
        # calculate the signals emerging from final output layer
        final_outputs = self.activation_function(final_inputs)
        
        return final_outputs

In [15]:
# number of input, hidden and output nodes
input_nodes = 83625
hidden_nodes = 15000
output_nodes = 5

# learning rate
learning_rate = 0.35

# create instance of neural network
n = neuralNetwork(input_nodes,hidden_nodes,output_nodes, learning_rate)

In [16]:
#load the mnist training data CSV file into a list
training_data_file=open("train_250_20220531.csv",'r')
training_data_list=training_data_file.readlines()
training_data_file.close()

len(training_data_list)


250

In [17]:
'''
#train the neural network
#go through all records on the training data set
for record in training_data_list:
     #split the record by the ',' commas
     all_values=record.split(',')
     #scale and shift the inouts
     inputs=(numpy.asfarray(all_values[1:])/255.0*0.99)+0.01
     #create the target outout values (all 0.01, except the desired label which is 0.99)
     targets=numpy.zeros(output_nodes)+0.01
     #all_values[0] is the target label for this record
     targets[int(all_values[0])]=0.99
     n.train(inputs,targets)
     pass
'''

"\n#train the neural network\n#go through all records on the training data set\nfor record in training_data_list:\n     #split the record by the ',' commas\n     all_values=record.split(',')\n     #scale and shift the inouts\n     inputs=(numpy.asfarray(all_values[1:])/255.0*0.99)+0.01\n     #create the target outout values (all 0.01, except the desired label which is 0.99)\n     targets=numpy.zeros(output_nodes)+0.01\n     #all_values[0] is the target label for this record\n     targets[int(all_values[0])]=0.99\n     n.train(inputs,targets)\n     pass\n"

In [18]:
#epochs is the numbers of times the training data set is used for training

epochs=1
for e in range(epochs):
    #go through all records in the training data set
    for record in training_data_list:
        #split the record by the ',' commas
        all_values=record.split(',')
        #scale and shift the inputs
        inputs=(numpy.asfarray(all_values[1:])/255.0*0.99)+0.01
        #create the target output values (all 0.01, except the desired label which is 0.99)
        targets=numpy.zeros(output_nodes)+0.01
        #all_values[0] is the taeget label for this record
        targets[int(all_values[0])]=0.99
        n.train(inputs,targets)
        pass
    pass

In [19]:
#load the mnist test data CSV file into a list
test_data_file=open("test_25_20220531.csv",'r')
test_data_list=test_data_file.readlines()
test_data_file.close()

In [20]:
#test the neural network

#scorecard for how well the network performs, initially empty
scorecard=[]

output_list = []

#go through all the records in the test data set
for record in test_data_list:
    #split the record by the ',' commas
    all_values=record.split(',')
    #correct answer is first value
    correct_label=int(all_values[0])
    print(correct_label,"correct label")
    #scale and shift the inputs
    inputs=(numpy.asfarray(all_values[1:])/255.0*0.99)+0.01
    #query the network
    outputs=n.query(inputs)
    
    output_list.append(outputs)
    
    label=numpy.argmax(outputs)
    print(label,"guessed answer")
    #append correct or incorrect to list
    if (label==correct_label):
        #network's answer matches correct answer, add 1 to scorecard
        scorecard.append(1)
    else:
        #network's answer does not match correct answer, add 0 to scorecard
        scorecard.append(0)
        pass
    pass

1 correct label
4 guessed answer
4 correct label
4 guessed answer
1 correct label
4 guessed answer
3 correct label
4 guessed answer
4 correct label
4 guessed answer
2 correct label
4 guessed answer
1 correct label
4 guessed answer
2 correct label
4 guessed answer
0 correct label
4 guessed answer
4 correct label
4 guessed answer
0 correct label
4 guessed answer
3 correct label
4 guessed answer
2 correct label
4 guessed answer
1 correct label
4 guessed answer
4 correct label
4 guessed answer
0 correct label
4 guessed answer
3 correct label
4 guessed answer
0 correct label
4 guessed answer
4 correct label
4 guessed answer
1 correct label
4 guessed answer
2 correct label
4 guessed answer
3 correct label
4 guessed answer
0 correct label
4 guessed answer
3 correct label
4 guessed answer
2 correct label
4 guessed answer


In [21]:
#calculate the performance score, the fraction of correct answers
scorecard_array=numpy.asarray(scorecard)
print("performance=",scorecard_array.sum()/scorecard_array.size)
#hidden : 5000     epoch:1     LR:0.35

performance= 0.2


In [22]:
print(outputs)
print(output_list)

[[3.80216032e-30]
 [1.25176244e-17]
 [1.18268393e-27]
 [5.52716908e-36]
 [1.00000000e+00]]
[array([[3.64868214e-30],
       [1.20689122e-17],
       [1.13468342e-27],
       [5.29563905e-36],
       [1.00000000e+00]]), array([[3.44533887e-30],
       [1.15553097e-17],
       [1.06478066e-27],
       [4.94487084e-36],
       [1.00000000e+00]]), array([[3.64960734e-30],
       [1.20802722e-17],
       [1.13564990e-27],
       [5.29484926e-36],
       [1.00000000e+00]]), array([[3.65355269e-30],
       [1.19794223e-17],
       [1.12742831e-27],
       [5.30288653e-36],
       [1.00000000e+00]]), array([[3.50316832e-30],
       [1.16572678e-17],
       [1.08282268e-27],
       [5.05416121e-36],
       [1.00000000e+00]]), array([[3.79734168e-30],
       [1.25190136e-17],
       [1.18325919e-27],
       [5.52089429e-36],
       [1.00000000e+00]]), array([[3.65269979e-30],
       [1.20838094e-17],
       [1.13664955e-27],
       [5.29968580e-36],
       [1.00000000e+00]]), array([[3.81728299e